<a href="https://colab.research.google.com/github/eisbetterthanpi/hypergraph/blob/main/hgnn_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title head
# https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/sparse/hgnn.ipynb
# https://github.com/dmlc/dgl/blob/master/notebooks/sparse/hgnn.ipynb
# https://github.com/dmlc/dgl/blob/master/examples/sparse/hgnn.py
import torch

cite=torch.Tensor([[0, 1, 2, 2, 2, 2, 3, 4, 5, 5, 5, 5, 6, 7, 7, 8, 8, 9, 9, 10],
                    [0, 0, 0, 1, 3, 4, 2, 1, 0, 2, 3, 4, 2, 1, 3, 1, 3, 2, 4, 4]])
H = torch.sparse_coo_tensor(indices=cite, values=torch.ones(cite.shape[1]),).coalesce()
# uncoalesced tensors, may be duplicate coords in the indices; in this case, the interpretation is that the value at that index is the sum of all duplicate value entries
# vert _ is in hyperedge _
print(H.to_dense()) # cols: hyperedges ; rows: verts

# node_degrees = H.sum(1)
# print("Node degrees", node_degrees)
# hyperedge_degrees = H.sum(0)
# print("Hyperedge degrees", hyperedge_degrees.values())



## Hypergraph Neural Network (HGNN) Layer

The [HGNN layer](https://arxiv.org/pdf/1809.09401.pdf) is defined as:

$$f(X^{(l)}, H; W^{(l)}) = \sigma(L X^{(l)} W^{(l)})$$$$L = D_v^{-1/2} H B D_e^{-1} H^\top D_v^{-1/2}$$

where

* $H \in \mathbb{R}^{N \times M}$ is the incidence matrix of hypergraph with $N$ nodes and $M$ hyperedges.
* $D_v \in \mathbb{R}^{N \times N}$ is a diagonal matrix representing node degrees, whose $i$-th diagonal element is $\sum_{j=1}^M H_{ij}$.
* $D_e \in \mathbb{R}^{M \times M}$ is a diagonal matrix representing hyperedge degrees, whose $j$-th diagonal element is $\sum_{i=1}^N H_{ij}$.
* $B \in \mathbb{R}^{M \times M}$ is a diagonal matrix representing the hyperedge weights, whose $j$-th diagonal element is the weight of $j$-th hyperedge.  In our example, $B$ is an identity matrix.

The following code builds a two-layer HGNN.

In [7]:
# @title requests data
import requests
url = 'https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz'
# response = requests.get(url)
open("cora.tgz", "wb").write(response.content)

import tarfile # os, sys,
tar = tarfile.open('cora.tgz', 'r')
tar.extractall('/content')

import torch

content = open("cora/cora.content", "r")
# print(content.read(10000))
# paper id, bag of words bool, category 0-6 # all str
rlst = content.read().split('\n')[:-1] # bec last row is ''
pid = [] # paper id
bow = [] # bag of words
cls = [] # classes
# category: Case_Based, Genetic_Algorithms, Neural_Networks, Probabilistic_Methods, Reinforcement_Learning, Rule_Learning, Theory
category = {'Case_Based':0, 'Genetic_Algorithms':1, 'Neural_Networks':2, 'Probabilistic_Methods':3, 'Reinforcement_Learning':4, 'Rule_Learning':5, 'Theory':6} # cora
for r in rlst:
    rr=r.split('\t')
    pid.append(int(rr[0]))
    bow.append(list(map(float, rr[1:-1]))) # must be float
    cls.append(category[rr[-1]])
pid=torch.tensor(pid)
X=torch.tensor(bow)
Y=torch.tensor(cls)
num_classes=7

# https://stellargraph.readthedocs.io/en/v1.0.0rc1/demos/node-classification/gcn/gcn-cora-node-classification-example.html
# The Cora dataset consists of 2708 scientific publications
# classified into one of seven classes.
# The citation network consists of 5429 links
# Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary.
# The dictionary consists of 1433 unique words

cites = open("cora/cora.cites", "r") # cite relation
clst = cites.read().split('\n')[:-1] # bec last row is ''
cite = [] #
for c in clst:
    cc=c.split('\t')
    cite.append([int(cc[0]),int(cc[1])])
cite=torch.tensor(cite) # [5429]

ukeys = torch.unique(pid)
uvals = torch.arange(len(ukeys))
udict = dict(zip(ukeys.tolist(), uvals.tolist())) # assign new id to each paper
pid = pid.apply_(udict.get)
cite = cite.apply_(udict.get)

num_v = len(pid)
H = torch.sparse_coo_tensor(indices=cite.T, values=torch.ones(cite.shape[0]), size=(num_v, num_v)).coalesce() # size=(2708, 2708), nnz=5429, layout=torch.sparse_coo
id = torch.sparse.spdiags(torch.ones(H.shape[0]),torch.tensor(0),H.shape)
H = (id + H).coalesce() # each vert got its hyperedge, contain all cited and itself, [2708, 2708], incedence matrix, |V| hyperedges

train_mask, val_mask, test_mask = torch.zeros(3, num_v, dtype=torch.bool)
train_mask[:140], val_mask[140:640], test_mask[-1000:] = True, True, True # cora mask
# print(len(train_mask))
# print(train_mask)
# H, X, Y, num_classes, train_mask, val_mask, test_mask = load_data()

# print(train_mask, val_mask, test_mask)
# print(sum(train_mask), sum(val_mask), sum(test_mask)) # 140), (500), (1000)
# print(sum(test_mask[-1000:]))
# print(len(test_mask)) # 2708
# print(train_mask[140])
# [:140], [140:640], [-1000:]

# print(H.shape, X.shape, Y.shape) # [2708, 2708], [2708, 1433], [2708]


In [ ]:
# @title edge/ incidence list

# edic = dict((id, [id]) for id in pid.tolist()) # edge list H(E)={e1,e2,e3}={{A,D},{D,E},{A,B,C}}
# idic = dict((id, [id]) for id in pid.tolist()) # incidence list {A:{e1,e3}, B:{e3}, C:{e3}, D:{e1,e2}, E:{e2}}
elst = [[id] for id in pid.tolist()] # edge list H(E)={e1,e2,e3}={{A,D},{D,E},{A,B,C}}
ilst = [[id] for id in pid.tolist()] # incidence list {A:{e1,e3}, B:{e3}, C:{e3}, D:{e1,e2}, E:{e2}}
for a,b in cite.tolist():
    elst[a].append(b)
    ilst[b].append(a)
elst = torch.tensor(elst)
ilst = torch.tensor(ilst)
# print(elst)
# print(ilst)



In [6]:
mask = torch.rand(10) <= 0.7
print(mask)
# r=torch.rand(10)
# print(r*mask)
# print(r[~mask])
# print(r[mask==True])
# r=[i for i in range(10)]
# print([rr for rr, m in zip(r, mask) if m])

a=[0,1,2,3]
# print(a[mask])
print(mask[a])
print([aa for aa, m in zip(a, mask[a]) if m])


tensor([ True, False, False,  True, False, False,  True,  True,  True, False])
tensor([ True, False, False,  True])
[0, 3]


In [ ]:
# @title dgl data
!pip install dgl

# "co-cite" relationship: hyperedge includes all the other papers it cited, as well as the paper itself
# then incidence mat = incidence mat + id
import torch
from dgl.data import CoraGraphDataset # https://github.com/dmlc/dgl/blob/master/python/dgl/data/citation_graph.py

def load_data():
    dataset = CoraGraphDataset()
    graph = dataset[0]
    indices = torch.stack(graph.edges())
    H = torch.sparse_coo_tensor(indices=indices, values=torch.ones(indices.shape[1]),).coalesce()
    id = torch.sparse.spdiags(torch.ones(H.shape[0]),torch.tensor(0),H.shape) # torch.eye(H.shape[0])
    H = (id + H).coalesce() # each vert got its hyperedge, contain all cited and itself, [2708, 2708], incedence matrix, |V| hyperedges

    X = graph.ndata["feat"] #[2708, 1433] num papers, len bag of words
    Y = graph.ndata["label"] # [2708], classiifcation 0-6
    train_mask = graph.ndata["train_mask"]
    val_mask = graph.ndata["val_mask"]
    test_mask = graph.ndata["test_mask"]
    return H, X, Y, dataset.num_classes, train_mask, val_mask, test_mask

H, X, Y, num_classes, train_mask, val_mask, test_mask = load_data()
# print(H.shape, X.shape, Y.shape) # [2708, 2708], [2708, 1433], [2708]


In [83]:
# @title HMPNN elst, ilst
import torch
import torch.nn as nn
# Vert msg = fv(vert ebd)
# Edge msg = fw(edge emb, Sum Vert msgs)
# Vert emb1 = gv(vert emb, Sum edge msgs)
# Edge emb1 = gw(edge emb, Sum Vert msgs)

class MsgPass(nn.Module):
    def __init__(self, vembdim, eembdim, vmsgdim, emsgdim):
        super(MsgPass, self).__init__()
        self.h_dim = 4
        self.fv = nn.Sequential(
            # nn.Linear(vembdim, self.h_dim), nn.ReLU(),
            nn.Linear(vembdim, self.h_dim), nn.Sigmoid(),
            nn.Linear(self.h_dim, vmsgdim),
            # nn.Linear(vembdim, vmsgdim), nn.ReLU(),
            # nn.Linear(vembdim, vmsgdim), nn.Sigmoid(),
            )
        self.fw = nn.Sequential(
            # nn.Linear(eembdim+vmsgdim, self.h_dim), nn.ReLU(),
            nn.Linear(eembdim+vmsgdim, self.h_dim), nn.Sigmoid(),
            nn.Linear(self.h_dim, emsgdim),
            # nn.Linear(eembdim+vmsgdim, emsgdim), nn.Sigmoid(),
            # nn.Linear(eembdim+vmsgdim, emsgdim), nn.ReLU(),
            )
        self.gv = nn.Sequential(
            # nn.Linear(vembdim+emsgdim, self.h_dim), nn.ReLU(),
            nn.Linear(vembdim+emsgdim, self.h_dim), nn.Sigmoid(),
            nn.Linear(self.h_dim, vembdim),
            # nn.Linear(vembdim+emsgdim, vembdim), nn.ReLU(),
            # nn.Linear(vembdim+emsgdim, vembdim), nn.Sigmoid(),
            )
        self.gw = nn.Sequential(
            # nn.Linear(eembdim+vmsgdim, self.h_dim), nn.ReLU(),
            nn.Linear(eembdim+vmsgdim, self.h_dim), nn.Sigmoid(),
            nn.Linear(self.h_dim, eembdim),
            # nn.Linear(eembdim+vmsgdim, eembdim), nn.Sigmoid(),
            # nn.Linear(eembdim+vmsgdim, eembdim), nn.ReLU(),
            )
        self.vmsgdim = vmsgdim
        self.emsgdim = emsgdim

    def forward(self, vemb, eemb, elst, ilst):
        # semsg=torch.stack([torch.mean(emsg[e],0) for e in ilst]) # given v, get all emsgs then aggregate
        vmsg = self.fv(vemb)
        # vmsg = self.fv(torch.cat((vemb, semsg), 1))
        vmsg = F.dropout(F.batch_norm(vmsg,torch.zeros(self.vmsgdim),torch.ones(self.vmsgdim)),p=0.5)

        mvemb, meemb, melst, milst = vemb, eemb, elst, ilst
        _, meemb, melst, milst = adjdrop(vemb, eemb, elst, ilst, p=0.7)

        svmsg=torch.stack([torch.sum(vmsg[v],0) for v in melst]) # given e, get all vmsgs then aggregate
        memsg = self.fw(torch.cat((meemb, svmsg), 1))
        memsg = F.dropout(F.batch_norm(memsg,torch.zeros(self.emsgdim),torch.ones(self.emsgdim)),p=0.5)

        # cannot be mean bec vert in ilst may be isolated, divide by 0 hyperedges
        semsg=torch.stack([torch.sum(memsg[e],0) for e in milst]) # given v, get all emsgs then aggregate
        vemb1 = self.gv(torch.cat((vemb, semsg), 1))

        svmsg=torch.stack([torch.sum(vmsg[v],0) for v in elst]) # given e, get all vmsgs then aggregate
        eemb1 = self.gw(torch.cat((eemb, svmsg), 1))
        return vemb1, eemb1

def ilst_from_elst(elst, n_v=len(ilst)): # generate incidence list from edge list
    ilst = [[] for id in range(n_v)]
    for e,vs in enumerate(elst):
        [ilst[v].append(e) for v in vs]
    return ilst

def adjdrop(vemb, eemb, elst, ilst, p=0.7): # adjacency dropout, maybe can replace with slicing of sparse tensors if pytorch implements it
    mask = torch.rand(len(elst)) >= p # True->keep, throw p
    melst = [e for e, m in zip(elst, mask) if m]
    meemb = eemb[mask==True]
    milst = ilst_from_elst(melst)
    return vemb, meemb, melst, milst


class HMPNN(nn.Module):
    def __init__(self, outdim, vembdim, eembdim):
        super(HMPNN, self).__init__()
        self.msgpass = MsgPass(vembdim, eembdim, vmsgdim=2, emsgdim=2)
        self.lin = nn.Linear(vembdim, outdim)

    def forward(self, x, elst=elst, ilst=ilst):
        vemb = x
        eemb = torch.zeros(len(elst),eembdim)
        # vemb, eemb = self.msgpass(vemb, eemb)
        vemb, eemb = self.msgpass(vemb, eemb, elst=elst, ilst=ilst)
        vemb, eemb = self.msgpass(vemb, eemb, elst=elst, ilst=ilst)
        x = self.lin(vemb)
        return x

num_v,vembdim=X.shape
eembdim=2
num_classes=7
model=HMPNN(num_classes, vembdim, eembdim)
yhat = model(X, elst, ilst)
# print(yhat.shape) # [2708, 7]

# print(len(X[0]))

In [ ]:
# @title models
import torch
import torch.nn as nn
import torch.nn.functional as F

@torch.no_grad
def hypergraph_laplacian(H):
    N,M = H.shape
    d_V = H.sum(1).to_dense() # node deg
    d_E = H.sum(0).to_dense() # edge deg
    D_v_invsqrt = torch.sparse.spdiags(d_V**-0.5,torch.tensor(0),(N,N)) # torch.diag(d_V**-0.5)
    D_e_inv = torch.sparse.spdiags(d_E**-1,torch.tensor(0),(M,M)) # torch.diag(d_E**-1)
    B = torch.sparse.spdiags(torch.ones(M),torch.tensor(0),(M,M)) # torch.eye(M) # B is id, dim n_edges
    return D_v_invsqrt @ H @ B @ D_e_inv @ H.T @ D_v_invsqrt # Laplacian

class HGNN(nn.Module): # https://github.com/dmlc/dgl/blob/master/notebooks/sparse/hgnn.ipynb
    def __init__(self, H, in_size, out_size, hidden_dims=16):
        super().__init__()
        self.W1 = nn.Linear(in_size, hidden_dims)
        self.W2 = nn.Linear(hidden_dims, out_size)
        self.dropout = nn.Dropout(0.5)
        self.L = hypergraph_laplacian(H)

    def forward(self, H, X):
        X = self.L @ self.W1(self.dropout(X))
        X = F.relu(X)
        X = self.L @ self.W2(self.dropout(X))
        return X

# Hypergraph Convolution and Hypergraph Attention https://arxiv.org/pdf/1901.08150.pdf
class HypergraphAttention(nn.Module): # https://github.com/dmlc/dgl/blob/master/examples/sparse/hypergraphatt.py
    def __init__(self, in_size, out_size):
        super().__init__()
        self.P = nn.Linear(in_size, out_size)
        self.a = nn.Linear(2 * out_size, 1)

    def forward(self, H, X, X_edges):
        Z = self.P(X)
        Z_edges = self.P(X_edges)
        # sim = self.a(torch.cat([Z[H.indices()[0]], Z_edges[H.indices()[1]]], 1))
        sim = self.a(torch.cat([Z[H.coalesce().indices()[0]], Z_edges[H.coalesce().indices()[1]]], 1))
        sim = F.leaky_relu(sim, 0.2).squeeze(1)
        print(H.shape,sim.shape) # [2708, 2708]) torch.Size([5429] og[13264]
        # H_att = torch.sparse_coo_tensor(indices=H.indices(), values=sim,).coalesce()
        H_att = torch.sparse_coo_tensor(indices=H.coalesce().indices(), values=sim,).coalesce()
        H_att = torch.sparse.softmax(H_att,1)
        # print(H.shape,H_att.shape) # [2708, 2708], [1898, 2708]
        # print(hypergraph_laplacian(H_att).shape, Z.shape)
        return hypergraph_laplacian(H_att) @ Z

class Net(nn.Module):
    def __init__(self, in_size, out_size, hidden_size=16):
        super().__init__()
        self.layer1 = HypergraphAttention(in_size, hidden_size)
        self.layer2 = HypergraphAttention(hidden_size, out_size)

    def forward(self, H, X):
        Z = self.layer1(H, X, X)
        Z = F.elu(Z)
        Z = self.layer2(H, Z, Z)
        return Z



In [41]:
# @title train/ eval
import torch
import torch.nn.functional as F

def train(model, optimizer, H, X, Y, train_mask):
    model.train()
    Y_hat = model(H, X)
    loss = F.cross_entropy(Y_hat[train_mask], Y[train_mask]) # loss_fn = nn.CrossEntropyLoss()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def accuracy(yhat, y): return (yhat.argmax(1) == y).type(torch.float).sum().item()/y.shape[0]

def evaluate(model, H, X, Y, val_mask, test_mask):
    model.eval()
    Y_hat = model(H, X) # model(X)
    val_acc = accuracy(Y_hat[val_mask], Y[val_mask])
    test_acc = accuracy(Y_hat[test_mask], Y[test_mask])
    return val_acc, test_acc


def trainl(model, optimizer, elst, ilst, X, Y, train_mask):
    model.train()
    Y_hat = model(X,elst, ilst)
    l,r=0,4
    # print(Y_hat[train_mask][l:r], Y[train_mask][l:r])
    loss = F.cross_entropy(Y_hat[train_mask], Y[train_mask]) # loss_fn = nn.CrossEntropyLoss()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluatel(model, elst, ilst, X, Y, val_mask, test_mask):
    model.eval()
    Y_hat = model(X,elst, ilst) # model(X)
    val_acc = accuracy(Y_hat[val_mask], Y[val_mask])
    test_acc = accuracy(Y_hat[test_mask], Y[test_mask])
    return val_acc, test_acc

# val_acc, test_acc = evaluatel(model, elst, ilst, X, Y, val_mask, test_mask)
# val_acc, test_acc = evaluatel(model, elst, ilst, X, Y, val_mask, train_mask)
# print(val_acc, test_acc)



In [ ]:
# @title run

# model = HGNN(H, X.shape[1], num_classes) # hg conv
# model = Net(X.shape[1], num_classes) # hg att
model = HMPNN(num_classes, vembdim, eembdim) # hg msg pass
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # 0.001
# optimizer.lr=0.01

for epoch in range(200):
    # loss = train(model, optimizer, H, X, Y, train_mask)
    # val_acc, test_acc = evaluate(model, H, X, Y, val_mask, test_mask)
    loss = trainl(model, optimizer, elst, ilst, X, Y, train_mask)
    val_acc, test_acc = evaluatel(model, elst, ilst, X, Y, val_mask, test_mask)
    print(f"test loss: {loss:.5f}, Val acc: {val_acc:.5f}, Test acc: {test_acc:.5f}")

# attn 200epoch 1min test loss: 0.00641, Val acc: 0.76000, Test acc: 0.74700
# hmpnn relu 7 epoch test loss: 0.00779, Val acc: 0.53800, Test acc: 0.47500
# hmpnn relu 200 epoch test loss: 0.00000, Val acc: 0.43400, Test acc: 0.41000
# hmpnn 2lin relu 200 epoch test loss: 0.02837, Val acc: 0.44000, Test acc: 0.38000
# hmpnn sigmoid test loss: 0.00064, Val acc: 0.53000, Test acc: 0.45100
# 2hmpnn sigmoid test loss: 1.81603, Val acc: 0.35000, Test acc: 0.29500
# 2hmpnn relu test loss: 0.00000, Val acc: 0.42800, Test acc: 0.45100
# 2hmpnn 2lin relu 200 epoch test loss: 1.81581, Val acc: 0.35000, Test acc: 0.29500
# 2hmpnn 2lin sigmoid 200 epoch test loss: 1.81585, Val acc: 0.35000, Test acc: 0.29500




### trash

In [ ]:
# @title hmpnn with H rej
import torch
import torch.nn as nn
# Vert msg = fv(vert ebd)
# Edge msg = fw(edge emb, Sum Vert msgs)
# Vert emb1 = gv(vert emb, Sum edge msgs)
# Edge emb1 = gw(edge emb, Sum Vert msgs)

class MsgPass(nn.Module):
    def __init__(self, vembdim, eembdim, vmsgdim, emsgdim):
        super(MsgPass, self).__init__()
        self.fv = nn.Sequential(
            # nn.Linear(vembdim, 64), nn.ReLU(),
            nn.Linear(vembdim, vmsgdim), nn.ReLU(),
            # nn.Linear(64, vmsgdim),
            )
        self.fw = nn.Sequential(
            # nn.Linear(eembdim+vmsgdim, 64), nn.ReLU(),
            nn.Linear(eembdim+vmsgdim, emsgdim), nn.ReLU(),
            # nn.Linear(64, emsgdim),
            )
        self.gv = nn.Sequential(
            # nn.Linear(vembdim+emsgdim, 64), nn.ReLU(),
            nn.Linear(vembdim+emsgdim, vembdim), nn.ReLU(),
            # nn.Linear(64, vembdim),
            )
        self.gw = nn.Sequential(
            # nn.Linear(eembdim+vmsgdim, 64), nn.ReLU(),
            nn.Linear(eembdim+vmsgdim, eembdim), nn.ReLU(),
            # nn.Linear(64, eembdim),
            )

    def forward(self, H, vemb, eemb):
        # semsg=torch.stack([torch.mean(emsg[e],0) for e in ilst]) # given v, get all emsgs then aggregate
        vmsg = self.fv(vemb)

        # vmsg = self.fv(torch.cat((vemb, semsg), 1))
        # print(vmsg.shape)
        mask = torch.rand(len(elst)) <= 0.7
        # melst = [e if m for e, m in zip(elst, mask)]
        melst = [e if m else [] for e, m in zip(elst, mask)]
        print(melst)
        # svmsg=torch.stack([torch.mean(vmsg[v],0) for v in elst]) # given e, get all vmsgs then aggregate
        svmsg=torch.stack([torch.mean(vmsg[v],0) for v in melst]) # given e, get all vmsgs then aggregate
        print(svmsg.shape)
        emsg = self.fw(torch.cat((eemb, svmsg), 1))
        # emsg = self.fw(torch.cat((eemb[mask==True], svmsg), 1))
        # nn.BatchNorm1d(dim) # nn.Dropout(p=0.7)

        semsg=torch.stack([torch.mean(emsg[e],0) for e in ilst]) # given v, get all emsgs then aggregate
        # semsg=torch.stack([torch.mean(emsg[e],0) if mask[i] else 0 for i,e in enumerate(ilst)]) # given v, get all emsgs then aggregate
        vemb1 = self.gv(torch.cat((vemb, semsg), 1))
        # nn.BatchNorm1d(dim) # nn.Dropout(p=0.7)

        # svmsg=torch.stack([torch.mean(vmsg[v],0) for v in elst]) # given e, get all vmsgs then aggregate
        # [aa for aa, m in zip(a, mask[a]) if m]
        svmsg=torch.stack([torch.mean(vmsg[v],0) for v in melst]) # given e, get all vmsgs then aggregate
        eemb1 = self.gw(torch.cat((eemb, svmsg), 1))
        print(svmsg.shape,eemb1.shape)
        return vemb1, eemb1

class HMPNN(nn.Module):
    def __init__(self, outdim, vembdim, eembdim):
        super(HMPNN, self).__init__()
        self.msgpass = MsgPass(vembdim, eembdim, vmsgdim=2, emsgdim=2)
        self.lin = nn.Linear(vembdim, outdim)

    def forward(self, H, X):
        vemb = x
        eemb = torch.zeros(len(elst),eembdim)
        # vemb, eemb = self.msgpass(vemb, eemb)
        vemb, eemb = self.msgpass(vemb, eemb, elst=elst, ilst=ilst)
        x = self.lin(vemb)
        return x

num_v,vembdim=X.shape
eembdim=2
num_classes=7
model=HMPNN(num_classes, vembdim, eembdim)
model(X, elst, ilst)


Y_hat = model(H, X)


# print(len(X[0]))

In [ ]:
# @title old og hg attn
Hypergraph Convolution and Hypergraph Attention
(https://arxiv.org/pdf/1901.08150.pdf).
import argparse
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from dgl.data import CoraGraphDataset
def accuracy(yhat, y): return (yhat.argmax(1) == y).type(torch.float).sum().item()/y.shape[0]


def hypergraph_laplacian(H):
    ###########################################################
    # (HIGHLIGHT) Compute the Laplacian with Sparse Matrix API
    ###########################################################
    d_V = H.sum(1)  # node degree
    d_E = H.sum(0)  # edge degree
    n_edges = d_E.shape[0]
    D_V_invsqrt = dglsp.diag(d_V**-0.5)  # D_V ** (-1/2)
    D_E_inv = dglsp.diag(d_E**-1)  # D_E ** (-1)
    W = dglsp.identity((n_edges, n_edges))
    return D_V_invsqrt @ H @ W @ D_E_inv @ H.T @ D_V_invsqrt


class HypergraphAttention(nn.Module):
    """Hypergraph Attention module as in the paper
    `Hypergraph Convolution and Hypergraph Attention
    <https://arxiv.org/pdf/1901.08150.pdf>`_.
    """

    def __init__(self, in_size, out_size):
        super().__init__()

        self.P = nn.Linear(in_size, out_size)
        self.a = nn.Linear(2 * out_size, 1)

    def forward(self, H, X, X_edges):
        Z = self.P(X)
        Z_edges = self.P(X_edges)
        # print("H",H.shape) # 2708, 2708
        # print("H.row,H.col",H.row.shape,H.col.shape) # H.row,H.col tensor([   0,    0,    0,  ..., 2707, 2707, 2707]) tensor([   0,  633, 1862,  ..., 1473, 2706, 2707]) # [13264], [13264]
        # print("Z[H.row], Z_edges[H.col]",Z[H.row], Z_edges[H.col].shape) # [13264, 16], [13264, 16]
        print(Z,Z_edges.shape) # [2708, 16], [2708, 16]

        sim = self.a(torch.cat([Z[H.row], Z_edges[H.col]], 1))
        sim = F.leaky_relu(sim, 0.2).squeeze(1)
        # Reassign the hypergraph new weights.
        H_att = dglsp.val_like(H, sim)
        H_att = H_att.softmax()
        return hypergraph_laplacian(H_att) @ Z


class Net(nn.Module):
    def __init__(self, in_size, out_size, hidden_size=16):
        super().__init__()
        self.layer1 = HypergraphAttention(in_size, hidden_size)
        self.layer2 = HypergraphAttention(hidden_size, out_size)

    def forward(self, H, X):
        Z = self.layer1(H, X, X)
        Z = F.elu(Z)
        Z = self.layer2(H, Z, Z)
        return Z


def train(model, optimizer, H, X, Y, train_mask):
    model.train()
    Y_hat = model(H, X)
    loss = F.cross_entropy(Y_hat[train_mask], Y[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()


def evaluate(model, H, X, Y, val_mask, test_mask, num_classes):
    model.eval()
    Y_hat = model(H, X)
    val_acc = accuracy(Y_hat[val_mask], Y[val_mask])
    test_acc = accuracy(Y_hat[test_mask], Y[test_mask])
    return val_acc, test_acc


def load_data():
    dataset = CoraGraphDataset()
    graph = dataset[0]
    indices = torch.stack(graph.edges())
    H = dglsp.spmatrix(indices)
    H = H + dglsp.identity(H.shape)
    X = graph.ndata["feat"]
    Y = graph.ndata["label"]
    train_mask = graph.ndata["train_mask"]
    val_mask = graph.ndata["val_mask"]
    test_mask = graph.ndata["test_mask"]
    return H, X, Y, dataset.num_classes, train_mask, val_mask, test_mask


H, X, Y, num_classes, train_mask, val_mask, test_mask = load_data()
model = Net(X.shape[1], num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

with tqdm.trange(2) as tq:
    for epoch in tq:
        loss = train(model, optimizer, H, X, Y, train_mask)
        val_acc, test_acc = evaluate(
            model, H, X, Y, val_mask, test_mask, num_classes
        )
        tq.set_postfix(
            {
                "Loss": f"{loss:.5f}",
                "Val acc": f"{val_acc:.5f}",
                "Test acc": f"{test_acc:.5f}",
            },
            refresh=False,
        )

print(f"Test acc: {test_acc:.3f}")

